# Optimize general GKP codes for distance, and get the fidelity

In [4]:
using Distributed
if nprocs()==1
    addprocs(length(Sys.cpu_info()); exeflags=`--project=$(Base.active_project())`)
end    
println("Number of processes = $(nprocs())")

In [5]:
@everywhere begin
    using LatticeAlgorithms
    using LinearAlgebra
    using BlockDiagonals
    using Optim
        
    function f(S_param)
        S = S_func([S_param[1:N]..., [0 for _ in 1:N^2]..., S_param[N+1:N+N^2]...])
        M = M0 * transpose(S)
        d = distance(M)
        return -d # minus because we minize the objective function 
    end
    
end

using JLD2 

function get_minimum_minimizer_list(S_param_0_list)
    return pmap(S_param_0_list) do S_param_0
        solver = NelderMead()  # SimulatedAnnealing() # NelderMead() 
        res = optimize(f, S_param_0, solver, Optim.Options(iterations=2000, show_trace=false, show_every=200))
        minimum = Optim.minimum(res)
        minimizer = Optim.minimizer(res)

        [minimum, minimizer]

    end;
    
end

num_trial = 1000;

# N=2

In [3]:
@everywhere begin
    N = 2
    code = "opt_$N"; 
    M0 = BlockDiagonal([[sqrt(2) 0; 0 sqrt(2)], [[1 0; 0 1] for _ in 2:N]...]);
    S_func = general_symplectic(N, method = "Bloch_Messiah") 
end

# num_trial = 1000;
r_max = 3;
θ_max = 2π; 
S_param_0_list = [[((1 / r_max) .+ (r_max - (1 / r_max)) * rand(N))..., (θ_max * rand(N^2))...]  for _ in 1:num_trial]; 

minimum_minimizer_list = @time get_minimum_minimizer_list(S_param_0_list) ; 

 59.237241 seconds (9.51 M allocations: 533.615 MiB, 0.35% gc time, 5.50% compilation time)


In [4]:
optimized_distance = minimum([minimum_minimizer_list[i][1] for i in 1:num_trial]) 

-2.5066282128414965

In [5]:
minimizer = minimum_minimizer_list[argmin([minimum_minimizer_list[i][1] for i in 1:num_trial])][2]  

6-element Vector{Float64}:
 0.6584789738546364
 0.6584789379697481
 0.9995127023929551
 7.428697686358381
 7.067655800660195
 3.5668765893978813

In [6]:
S_opt = S_func([minimizer[1:N]..., [0 for _ in 1:N^2]..., minimizer[N+1:N+N^2]...])
M_opt = M0 * transpose(S_opt)

4×4 Matrix{Float64}:
  1.30882    0.108394   1.42093   -0.506162
 -0.404532   0.350698   1.88902    0.380736
  1.33574   -0.269221   0.286048   0.247981
  1.00475    0.357911  -0.925478   0.0766462

In [ ]:
fn = "data/optimized_codes/distance_$(code)_$(num_trial).jld2" ; 

In [7]:
# # Uncomment this cell if you want to overwrite the data

# jldsave(fn; 
#     optimized_distance = optimized_distance,
#     minimizer = minimizer,
#     S_opt=S_opt,
#     M_opt=M_opt,    
#     num_trial=num_trial)

In [8]:
load(fn)

Dict{String, Any} with 5 entries:
  "M_opt"              => [1.30882 0.108394 1.42093 -0.506162; -0.404532 0.3506…
  "S_opt"              => [0.925478 -0.286048 1.33574 1.00475; 0.0766462 0.2479…
  "minimizer"          => [0.658479, 0.658479, 0.999513, 7.4287, 7.06766, 3.566…
  "optimized_distance" => -2.50663
  "num_trial"          => 1000

In [9]:
## Known best result is from D4
best_result = √(2π) * (N/2)^(1/4)

2.5066282746310002

# N=3

In [10]:
@everywhere begin
    N = 3
    code = "opt_$N"; 
    M0 = BlockDiagonal([[sqrt(2) 0; 0 sqrt(2)], [[1 0; 0 1] for _ in 2:N]...]);
    S_func = general_symplectic(N, method = "Bloch_Messiah") 
end

# num_trial = 1000;
r_max = 3;
θ_max = 2π; 
S_param_0_list = [[((1 / r_max) .+ (r_max - (1 / r_max)) * rand(N))..., (θ_max * rand(N^2))...]  for _ in 1:num_trial]; 

minimum_minimizer_list = @time get_minimum_minimizer_list(S_param_0_list) ; 

 90.144094 seconds (54.43 k allocations: 2.527 MiB)


In [11]:
optimized_distance = minimum([minimum_minimizer_list[i][1] for i in 1:num_trial])  # -2.6424990160263677

-2.6710507476626355

In [12]:
minimizer = minimum_minimizer_list[argmin([minimum_minimizer_list[i][1] for i in 1:num_trial])][2]  

12-element Vector{Float64}:
 0.7310554863685963
 0.746121645685197
 0.7491080990533483
 2.082230804447372
 3.21756673624585
 4.0836580349545715
 6.205173281615245
 2.701734241492325
 6.102598173629776
 6.413886025526753
 5.407506479464776
 3.573378437152211

In [13]:
S_opt = S_func([minimizer[1:N]..., [0 for _ in 1:N^2]..., minimizer[N+1:N+N^2]...])
M_opt = M0 * transpose(S_opt)

6×6 Matrix{Float64}:
 -1.97001    0.244552    0.666809  -0.155204   0.0430447   0.376757
 -1.05526   -0.456543    0.690203   0.149944  -1.6855      0.00962171
 -0.444733  -0.141088   -1.42223    0.185018   0.786906    0.165664
  0.608804  -0.103065   -0.822784  -0.319813  -0.741133    0.175896
 -0.369226   0.244496   -1.02599    0.108822  -1.28139    -0.109934
 -1.05501   -0.0855669  -0.483938  -0.230712   0.491812   -0.286426

In [ ]:
fn = "data/optimized_codes/distance_$(code)_$(num_trial).jld2" ; 

In [ ]:
# # Uncomment this cell if you want to overwrite the data

# jldsave(fn; 
#     optimized_distance = optimized_distance,
#     minimizer = minimizer,
#     S_opt=S_opt,
#     M_opt=M_opt,    
#     num_trial=num_trial)

In [15]:
load(fn)

Dict{String, Any} with 5 entries:
  "M_opt"              => [-1.97001 0.244552 … 0.0430447 0.376757; -1.05526 -0.…
  "S_opt"              => [-1.39301 -0.746178 … -0.369226 -1.05501; 0.172924 -0…
  "minimizer"          => [0.731055, 0.746122, 0.749108, 2.08223, 3.21757, 4.08…
  "optimized_distance" => -2.67105
  "num_trial"          => 1000

In [16]:
## No known best result

# N=4

In [17]:
@everywhere begin
    N = 4
    code = "opt_$N"; 
    M0 = BlockDiagonal([[sqrt(2) 0; 0 sqrt(2)], [[1 0; 0 1] for _ in 2:N]...]);
    S_func = general_symplectic(N, method = "Bloch_Messiah") 
end

# num_trial = 1000;
r_max = 3;
θ_max = 2π; 
S_param_0_list = [[((1 / r_max) .+ (r_max - (1 / r_max)) * rand(N))..., (θ_max * rand(N^2))...]  for _ in 1:num_trial]; 

minimum_minimizer_list = @time get_minimum_minimizer_list(S_param_0_list) ; 

216.551347 seconds (54.44 k allocations: 2.663 MiB)


In [18]:
optimized_distance = minimum([minimum_minimizer_list[i][1] for i in 1:num_trial]) 
# 2.839 with 10000 samples
# 2.859 with 30000 samples

-2.82515178218537

In [19]:
minimizer = minimum_minimizer_list[argmin([minimum_minimizer_list[i][1] for i in 1:num_trial])][2]  

20-element Vector{Float64}:
 1.163244254494251
 0.5512834150128066
 0.7650424084421334
 2.2591036512069596
 2.793599221748231
 4.191469824570051
 5.721511242340718
 3.1481040594650427
 2.102606185540268
 1.6520370042413364
 5.066654830211398
 5.635226589899807
 1.6252016634806814
 0.023143446040152525
 4.574166411302724
 0.42050477056627633
 5.044247947106983
 0.6074353028322423
 4.9926646543349
 1.3234035490088096

In [20]:
S_opt = S_func([minimizer[1:N]..., [0 for _ in 1:N^2]..., minimizer[N+1:N+N^2]...])
M_opt = M0 * transpose(S_opt)

8×8 Matrix{Float64}:
  0.638569   0.361384    0.690668   …  -0.129361    2.87509  -0.0173488
 -3.70126    0.0623486   0.0351058     -0.241233    1.59038   0.0313631
 -0.404581   0.0909072   0.251404       0.268085    2.41837  -0.026287
 -0.931063  -0.0395025   0.223714       0.295281    2.40976   0.026381
  0.595625   0.081159    0.352065       0.100987   -5.73443   0.0460545
 -0.831224   0.0581556  -0.819017   …   0.0855224  -4.22187  -0.0625544
  0.88217    0.0564165  -1.22697        0.0181362   4.17953   0.0276862
 -0.577813   0.0861333  -0.599435       0.0463874  -2.53802   0.0455926

In [21]:
fn = "data/optimized_codes/distance_$(code)_$(num_trial).jld2" ; 

In [ ]:
# # Uncomment this cell if you want to overwrite the data

# jldsave(fn; 
#     optimized_distance = optimized_distance,
#     minimizer = minimizer,
#     S_opt=S_opt,
#     M_opt=M_opt,    
#     num_trial=num_trial)

In [22]:
load(fn)

Dict{String, Any} with 5 entries:
  "M_opt"              => [0.638569 0.361384 … 2.87509 -0.0173488; -3.70126 0.0…
  "S_opt"              => [0.451537 -2.61719 … 0.88217 -0.577813; 0.255537 0.04…
  "minimizer"          => [1.16324, 0.551283, 0.765042, 2.2591, 2.7936, 4.19147…
  "optimized_distance" => -2.82515
  "num_trial"          => 1000

In [23]:
## known best result from tess_4N
best_result = √(2π) * (N/2)^(1/4)

2.98090017885818

# N=5

In [ ]:
@everywhere begin
    N = 5
    code = "opt_$N"; 
    M0 = BlockDiagonal([[sqrt(2) 0; 0 sqrt(2)], [[1 0; 0 1] for _ in 2:N]...]);
    S_func = general_symplectic(N, method = "Bloch_Messiah") 
end

# num_trial = 1000;
r_max = 3;
θ_max = 2π; 
S_param_0_list = [[((1 / r_max) .+ (r_max - (1 / r_max)) * rand(N))..., (θ_max * rand(N^2))...]  for _ in 1:num_trial]; 

minimum_minimizer_list = @time get_minimum_minimizer_list(S_param_0_list) ; 

In [ ]:
optimized_distance = minimum([minimum_minimizer_list[i][1] for i in 1:num_trial]) 

In [ ]:
minimizer = minimum_minimizer_list[argmin([minimum_minimizer_list[i][1] for i in 1:num_trial])][2]  

In [ ]:
S_opt = S_func([minimizer[1:N]..., [0 for _ in 1:N^2]..., minimizer[N+1:N+N^2]...])
M_opt = M0 * transpose(S_opt)

In [ ]:
fn = "data/optimized_codes/distance_$(code)_$(num_trial).jld2" ; 

In [ ]:
# # Uncomment this cell if you want to overwrite the data

# jldsave(fn; 
#     optimized_distance = optimized_distance,
#     minimizer = minimizer,
#     S_opt=S_opt,
#     M_opt=M_opt,    
#     num_trial=num_trial)

In [ ]:
load(fn)

In [ ]:
## known best result from [5,1,3]

# N=6

In [ ]:
@everywhere begin
    N = 6
    code = "opt_$N"; 
    M0 = BlockDiagonal([[sqrt(2) 0; 0 sqrt(2)], [[1 0; 0 1] for _ in 2:N]...]);
    S_func = general_symplectic(N, method = "Bloch_Messiah") 
end

# num_trial = 1000;
r_max = 3;
θ_max = 2π; 
S_param_0_list = [[((1 / r_max) .+ (r_max - (1 / r_max)) * rand(N))..., (θ_max * rand(N^2))...]  for _ in 1:num_trial]; 

minimum_minimizer_list = @time get_minimum_minimizer_list(S_param_0_list) ; 

In [ ]:
optimized_distance = minimum([minimum_minimizer_list[i][1] for i in 1:num_trial]) 

In [ ]:
minimizer = minimum_minimizer_list[argmin([minimum_minimizer_list[i][1] for i in 1:num_trial])][2]  

In [ ]:
S_opt = S_func([minimizer[1:N]..., [0 for _ in 1:N^2]..., minimizer[N+1:N+N^2]...])
M_opt = M0 * transpose(S_opt)

In [ ]:
fn = "data/optimized_codes/distance_$(code)_$(num_trial).jld2" ; 

In [ ]:
# # Uncomment this cell if you want to overwrite the data

# jldsave(fn; 
#     optimized_distance = optimized_distance,
#     minimizer = minimizer,
#     S_opt=S_opt,
#     M_opt=M_opt,    
#     num_trial=num_trial)

In [ ]:
load(fn)

In [ ]:
## known best result from tess_4N
best_result = √(2π) * (N/2)^(1/4)

# N=7

In [ ]:
@everywhere begin
    N = 7
    code = "opt_$N"; 
    M0 = BlockDiagonal([[sqrt(2) 0; 0 sqrt(2)], [[1 0; 0 1] for _ in 2:N]...]);
    S_func = general_symplectic(N, method = "Bloch_Messiah") 
end

# num_trial = 1000;
r_max = 3;
θ_max = 2π; 
S_param_0_list = [[((1 / r_max) .+ (r_max - (1 / r_max)) * rand(N))..., (θ_max * rand(N^2))...]  for _ in 1:num_trial]; 

minimum_minimizer_list = @time get_minimum_minimizer_list(S_param_0_list) ; 

In [ ]:
optimized_distance = minimum([minimum_minimizer_list[i][1] for i in 1:num_trial]) 

In [ ]:
minimizer = minimum_minimizer_list[argmin([minimum_minimizer_list[i][1] for i in 1:num_trial])][2]  

In [ ]:
S_opt = S_func([minimizer[1:N]..., [0 for _ in 1:N^2]..., minimizer[N+1:N+N^2]...])
M_opt = M0 * transpose(S_opt)

In [ ]:
fn = "data/optimized_codes/distance_$(code)_$(num_trial).jld2" ; 

In [ ]:
# # Uncomment this cell if you want to overwrite the data

# jldsave(fn; 
#     optimized_distance = optimized_distance,
#     minimizer = minimizer,
#     S_opt=S_opt,
#     M_opt=M_opt,    
#     num_trial=num_trial)

In [ ]:
load(fn)

In [ ]:
## known best result from [7,1,3]

# N=8

In [ ]:
@everywhere begin
    N = 8
    code = "opt_$N"; 
    M0 = BlockDiagonal([[sqrt(2) 0; 0 sqrt(2)], [[1 0; 0 1] for _ in 2:N]...]);
    S_func = general_symplectic(N, method = "Bloch_Messiah") 
end

# num_trial = 1000;
r_max = 3;
θ_max = 2π; 
S_param_0_list = [[((1 / r_max) .+ (r_max - (1 / r_max)) * rand(N))..., (θ_max * rand(N^2))...]  for _ in 1:num_trial]; 

minimum_minimizer_list = @time get_minimum_minimizer_list(S_param_0_list) ; 

In [ ]:
optimized_distance = minimum([minimum_minimizer_list[i][1] for i in 1:num_trial]) 

In [ ]:
minimizer = minimum_minimizer_list[argmin([minimum_minimizer_list[i][1] for i in 1:num_trial])][2]  

In [ ]:
S_opt = S_func([minimizer[1:N]..., [0 for _ in 1:N^2]..., minimizer[N+1:N+N^2]...])
M_opt = M0 * transpose(S_opt)

In [ ]:
fn = "data/optimized_codes/distance_$(code)_$(num_trial).jld2" ; 

In [ ]:
# # Uncomment this cell if you want to overwrite the data

# jldsave(fn; 
#     optimized_distance = optimized_distance,
#     minimizer = minimizer,
#     S_opt=S_opt,
#     M_opt=M_opt,    
#     num_trial=num_trial)

In [ ]:
load(fn)

In [ ]:
## known best result from tess_4N
best_result = √(2π) * (N/2)^(1/4)

# N=9

In [ ]:
@everywhere begin
    N = 9
    code = "opt_$N"; 
    M0 = BlockDiagonal([[sqrt(2) 0; 0 sqrt(2)], [[1 0; 0 1] for _ in 2:N]...]);
    S_func = general_symplectic(N, method = "Bloch_Messiah") 
end

# num_trial = 1000;
r_max = 3;
θ_max = 2π; 
S_param_0_list = [[((1 / r_max) .+ (r_max - (1 / r_max)) * rand(N))..., (θ_max * rand(N^2))...]  for _ in 1:num_trial]; 

minimum_minimizer_list = @time get_minimum_minimizer_list(S_param_0_list) ; 

In [ ]:
optimized_distance = minimum([minimum_minimizer_list[i][1] for i in 1:num_trial]) 

In [ ]:
minimizer = minimum_minimizer_list[argmin([minimum_minimizer_list[i][1] for i in 1:num_trial])][2]  

In [ ]:
S_opt = S_func([minimizer[1:N]..., [0 for _ in 1:N^2]..., minimizer[N+1:N+N^2]...])
M_opt = M0 * transpose(S_opt)

In [ ]:
fn = "data/optimized_codes/distance_$(code)_$(num_trial).jld2" ; 

In [ ]:
# # Uncomment this cell if you want to overwrite the data

# jldsave(fn; 
#     optimized_distance = optimized_distance,
#     minimizer = minimizer,
#     S_opt=S_opt,
#     M_opt=M_opt,    
#     num_trial=num_trial)

In [ ]:
load(fn)

In [ ]:
## known best result from d=3 surface code

# N=10

In [ ]:
@everywhere begin
    N = 10
    code = "opt_$N"; 
    M0 = BlockDiagonal([[sqrt(2) 0; 0 sqrt(2)], [[1 0; 0 1] for _ in 2:N]...]);
    S_func = general_symplectic(N, method = "Bloch_Messiah") 
end

# num_trial = 1000;
r_max = 3;
θ_max = 2π; 
S_param_0_list = [[((1 / r_max) .+ (r_max - (1 / r_max)) * rand(N))..., (θ_max * rand(N^2))...]  for _ in 1:num_trial]; 

minimum_minimizer_list = @time get_minimum_minimizer_list(S_param_0_list) ; 

In [ ]:
optimized_distance = minimum([minimum_minimizer_list[i][1] for i in 1:num_trial]) 

In [ ]:
minimizer = minimum_minimizer_list[argmin([minimum_minimizer_list[i][1] for i in 1:num_trial])][2]  

In [ ]:
S_opt = S_func([minimizer[1:N]..., [0 for _ in 1:N^2]..., minimizer[N+1:N+N^2]...])
M_opt = M0 * transpose(S_opt)

In [ ]:
fn = "data/optimized_codes/distance_$(code)_$(num_trial).jld2" ; 

In [ ]:
# # Uncomment this cell if you want to overwrite the data

# jldsave(fn; 
#     optimized_distance = optimized_distance,
#     minimizer = minimizer,
#     S_opt=S_opt,
#     M_opt=M_opt,    
#     num_trial=num_trial)

In [67]:
load(fn)

Dict{String, Any} with 5 entries:
  "M_opt"              => [0.69458 -0.252138 … -1.89719 0.0704011; 0.953008 0.1…
  "S_opt"              => [0.491142 0.673878 … -0.222413 0.0351977; -0.178288 0…
  "minimizer"          => [0.664824, 1.31715, 1.44488, 2.31008, 1.37858, 1.8551…
  "optimized_distance" => -3.54596
  "num_trial"          => 1000

In [68]:
fn

"distance_opt_10_1000.jld2"

In [69]:
## known best result from tess_4N and [5,1,3] x hex?
best_result = √(2π) * (N/2)^(1/4)

3.748283535444117

# Get the fidelities for the optimized codes

In [ ]:
## 
Nrange = 2:10
M_opt_list = []
for N in Nrange
    push!(M_opt_list, load("data/optimized_codes/distance_opt_$(N)_$(num_trial).jld2")["M_opt"])
end


ξ_list = []
fidelities = Dict()
for (ind, N) in enumerate(Nrange)
    for σ in σrange
        fidelities[(N, σ)] = [0, 0, 0, 0]
        for _ in 1 : num_samples
            push!(ξ_list, (N, σ, σ * randn(2N)))
        end
    end
end

@time outcomes = pmap(ξ_list) do (N, σ, ξ)
    ind = findall(x->x==N, Nrange)[1]
    M_opt = M_opt_list[ind]
    Mperp = GKP_logical_operator_generator_canonical(M_opt)    
    
    count = [0, 0, 0, 0]; # I,X,Z,Y
    y = closest_point(ξ, √(2π) * Mperp) # y = √(2π) * transpose(Mperp) * u
    u = inv(transpose(√(2π) * Mperp)) * y
    nx, nz = round.(Int, u[1:2])

    if mod(nx, 2) == 0 && mod(nz, 2) == 0
        count[1] += 1
    elseif mod(nx, 2) == 1 && mod(nz, 2) == 0
        count[2] += 1
    elseif mod(nx, 2) == 0 && mod(nz, 2) == 1
        count[3] += 1
    elseif mod(nx, 2) == 1 && mod(nz, 2) == 1
        count[4] += 1
    end
    
    return (N, σ, count/num_samples)
end

for (N, σ, count) in outcomes
    fidelities[(N, σ)] += count
end



In [ ]:
# # Uncomment this cell if you want to overwrite the data

# fn = "data/optimized_codes/fidelity_optimized_codes.jld2"
# jldsave(fn; 
#     Nrange=Nrange, 
#     σrange=σrange,
#     num_samples=num_samples,
#     M_opt_list=M_opt_list,
#     fidelities=fidelities
# )
